# Foundational Mathematics for Engineers

**Welcome!** This lesson is the theoretical heart of everything we do. The laws of physics and chemistry that govern our processes are written in the language of mathematics. To build powerful simulations, we must first be fluent in this language. This notebook will build your intuition for the core concepts of calculus and differential equations from an engineering perspective.

**Objective:** To develop a deep, practical understanding of derivatives, integrals, and Ordinary Differential Equations (ODEs) and how they are used to model the dynamics of engineering systems.

**Learning Goals:**
1.  Understand the **derivative** as an instantaneous rate of change and learn to approximate it numerically.
2.  Understand the **integral** as the accumulation of a quantity and learn to compute it numerically.
3.  Grasp that an **Ordinary Differential Equation (ODE)** is a mathematical statement of the "rules of change" for a system.
4.  Learn how a computer solves an ODE by starting with an initial condition and taking small steps.
5.  Combine these concepts to build an interactive simulation of a dynamic physical system.

## Part 1: The Derivative - The Language of Instantaneous Change

In engineering, we are constantly concerned with rates. How fast is a reaction proceeding? How quickly is a tank heating up? A derivative is the mathematical tool for describing an **instantaneous rate of change**.

Visually, the derivative of a function at a point is simply the **slope of the tangent line** at that point.

The notation for the derivative of a function $y$ with respect to a variable $x$ is: $$ \frac{dy}{dx} $$

### 1.1 Numerical Approximation of a Derivative

While calculus provides rules for finding exact derivatives, in computational work we often approximate them using data. The simplest way is the **finite difference method**, which uses the familiar "rise over run" formula over a small interval $h$.
$$ \frac{dy}{dx} \approx \frac{y(x+h) - y(x)}{h} $$

**Engineering Example:** Let's model the concentration of a reactant in a batch reactor. The concentration profile is given by $C_A(t) = C_{A0} e^{-kt}$. The reaction rate at any moment is the derivative of this profile, $\frac{dC_A}{dt}$. Let's calculate and plot both.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Setup ---
C_A0 = 2.0  # Initial concentration (mol/L)
k = 0.5     # Rate constant (1/min)
t = np.linspace(0, 10, 100) # Time vector

# Calculate the concentration profile
C_A = C_A0 * np.exp(-k * t)

# --- Numerical Differentiation ---
# We can use np.diff() which calculates the difference between adjacent elements
dC_A = np.diff(C_A)
dt = np.diff(t)
rate = dC_A / dt

# --- Plotting ---
fig, ax1 = plt.subplots(figsize=(12, 7))

# Plot concentration on the left axis
ax1.set_xlabel('Time (minutes)', fontsize=14)
ax1.set_ylabel('Concentration, $C_A$ (mol/L)', color='royalblue', fontsize=14)
ax1.plot(t, C_A, color='royalblue', linewidth=3, label='Concentration Profile')
ax1.tick_params(axis='y', labelcolor='royalblue')

# Plot the calculated rate on the right axis
ax2 = ax1.twinx()
ax2.set_ylabel('Reaction Rate, $dC_A/dt$', color='firebrick', fontsize=14)
# We plot the rate at the midpoint of each interval, so we need to adjust the time vector
ax2.plot(t[:-1], rate, color='firebrick', linestyle='--', label='Reaction Rate (Numerical)')
ax2.tick_params(axis='y', labelcolor='firebrick')

fig.suptitle('Concentration and its Derivative (Reaction Rate)', fontsize=16, weight='bold')
fig.legend()
plt.show()

print("Observe the relationship: The rate is most negative (fastest consumption) at t=0 where the concentration is highest. As concentration decreases, the rate approaches zero.")

## Part 2: The Integral - The Language of Accumulation

An integral represents the **accumulation or summation of a quantity**. Visually, a definite integral is the **area under the curve** of a function between two points.

$$ \text{Total Quantity} = \int_{a}^{b} f(x) \,dx $$
**Engineering Example:** If a pump provides a variable flow rate into a tank over time, $F(t)$, the total volume added to the tank after a time $T$ is the integral of the flow rate from 0 to $T$.

In [ ]:
from scipy.integrate import quad # 'quad' is the primary tool for numerical integration

# --- Setup ---
# Let's say the inlet flow rate varies sinusoidally around an average
def flow_rate(t):
    return 10 + 2 * np.sin(np.pi * t / 5) # m^3/hr

# Time range for plotting
time = np.linspace(0, 20, 200)
flows = flow_rate(time)

# --- Numerical Integration ---
# We want to find the total volume added between t=5 and t=15 hours
t_start = 5.0
t_end = 15.0
# quad returns two values: the result and an estimate of the error
total_volume, error_estimate = quad(flow_rate, t_start, t_end)

print(f"The total volume added between t={t_start} and t={t_end} hr is {total_volume:.2f} m^3.")
print(f"(Numerical error estimate: {error_estimate:.2e})")

# --- Plotting ---
plt.figure(figsize=(12, 6))
plt.plot(time, flows, 'b-', linewidth=3, label='Inlet Flow Rate, F(t)')

# Create a shaded region for the integration area
integration_time = np.linspace(t_start, t_end, 100)
integration_flows = flow_rate(integration_time)
plt.fill_between(integration_time, integration_flows, color='skyblue', alpha=0.5, label='Integrated Area (Total Volume)')

plt.title('Integrating a Variable Flow Rate to Find Total Volume', fontsize=16, weight='bold')
plt.xlabel('Time (hours)', fontsize=14)
plt.ylabel('Flow Rate (m$^3$/hr)', fontsize=14)
plt.legend()
plt.show()

## Part 3: Ordinary Differential Equations (ODEs) - The Rules of Change

This is the grand synthesis. An ODE is an equation that relates a function to its own derivatives. **This is the language of dynamic systems.**

An ODE is a **rule**. It doesn't tell you the state of your system directly. It tells you **how the state is changing at any given moment**, based on the current state.

**Example:** The model for a cooling cup of coffee is Newton's Law of Cooling:
$$ \frac{dT}{dt} = -k (T - T_{ambient}) $$
**In words, this rule says:** *"The rate of change of the coffee's temperature is proportional to the difference between its current temperature and the room's temperature."*

To solve this, we need a starting point: the **initial condition** (the temperature at $t=0$). The computer then solves it by taking tiny steps forward in time, using the ODE as its guide for which direction to step.

### Interactive Capstone: The Leaky, Heated Tank
Let's model a tank that is being filled while also having a leak. At the same time, the incoming fluid is at a different temperature.

This system is described by two coupled ODEs based on mass and energy balances:

**1. Mass Balance (gives the change in liquid level, $h$):**
$$ \frac{dh}{dt} = \frac{1}{A} (F_{in} - F_{out}) $$
Where the outlet flow (leak) depends on the liquid level: $F_{out} = C_v \sqrt{h}$

**2. Energy Balance (gives the change in tank temperature, $T$):**
$$ \frac{dT}{dt} = \frac{F_{in}}{A \cdot h} (T_{in} - T) $$

In [ ]:
from scipy.integrate import solve_ivp
import ipywidgets as widgets
from IPython.display import display

def tank_model(t, y, F_in, T_in, A, Cv):
    """The ODE system for the leaky, heated tank."""
    h, T = y # Unpack the state vector [level, temperature]
    
    # Prevent sqrt(negative) if h is driven to zero by the solver
    if h < 0:
        h = 0
        
    # Calculate the outlet flow based on the current level
    F_out = Cv * np.sqrt(h)
    
    # ODE for level (mass balance)
    dh_dt = (F_in - F_out) / A
    
    # ODE for temperature (energy balance)
    # Add a safeguard for h=0 to prevent division by zero
    if h < 1e-6:
        dT_dt = 0
    else:
        dT_dt = (F_in / (A * h)) * (T_in - T)
        
    return [dh_dt, dT_dt]

def solve_and_plot(F_in=1.0, T_in=80.0, Cv=0.5):
    """Wraps the solver and plotting for the interactive widget."""
    A = 1.0 # m^2
    t_span = (0, 30) # minutes
    y0 = [0.1, 20.0] # Initial conditions: h=0.1m, T=20C
    
    # Use a lambda function to pass extra arguments to the model
    solution = solve_ivp(lambda t, y: tank_model(t, y, F_in, T_in, A, Cv), 
                         t_span, y0, dense_output=True)
    
    t_plot = np.linspace(t_span[0], t_span[1], 200)
    h_plot, T_plot = solution.sol(t_plot)
    
    # Plotting
    fig, ax1 = plt.subplots(figsize=(12, 7))
    ax1.set_xlabel('Time (minutes)', fontsize=14)
    ax1.set_ylabel('Tank Level (m)', fontsize=14, color='royalblue')
    ax1.plot(t_plot, h_plot, color='royalblue', linewidth=3, label='Level')
    ax1.tick_params(axis='y', labelcolor='royalblue')
    ax2 = ax1.twinx()
    ax2.set_ylabel('Temperature (°C)', fontsize=14, color='firebrick')
    ax2.plot(t_plot, T_plot, color='firebrick', linestyle='--', linewidth=3, label='Temperature')
    ax2.tick_params(axis='y', labelcolor='firebrick')
    fig.suptitle('Dynamic Simulation of a Leaky, Heated Tank', fontsize=16, weight='bold')
    fig.legend()
    plt.show()

# Create and display the interactive widget
interactive_plot = widgets.interactive(solve_and_plot,
    F_in=widgets.FloatSlider(value=1.0, min=0.5, max=2.0, step=0.1, description='Inlet Flow:'),
    T_in=widgets.FloatSlider(value=80.0, min=20.0, max=100.0, step=5, description='Inlet Temp:'),
    Cv=widgets.FloatSlider(value=0.5, min=0.1, max=1.0, step=0.05, description='Leak Valve Cv:'))

display(interactive_plot)

### Interpreting the Interactive Simulation

Play with the sliders to build your intuition!
*   Notice that the level doesn't rise forever. It reaches a **steady state** where the outlet flow from the leak exactly balances the inlet flow. This is a dynamic equilibrium.
*   If you increase the **Inlet Flow**, the steady-state level increases. Why?
*   If you increase the **Leak Valve Cv** (making the hole bigger), the steady-state level decreases. Why?
*   Observe how the temperature changes. It rises from its initial value and asymptotically approaches the inlet temperature. The rate of change is fastest at the beginning when the temperature difference ($T_{in} - T$) is largest.